# data processing

## json create

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# package import

from data_pack import DataSetDefinition
import utils_cpu
import utils_gpu

from typing import List, Dict
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [4]:
dataset_def = DataSetDefinition(
    processed_dir_suffix="128_v1"
)


In [5]:
dataset_def.processed_data_root_dir

'./data/processed'

In [6]:
step2_split = dataset_def.step2_dict_generator()

In [7]:
train_split = step2_split["train"]

In [8]:
from easydict import EasyDict
easy_dict = EasyDict(train_split[0])

In [9]:
easy_dict

{'image_path': './data/processed/train/images/obj_01_car/obj_01_car_004_CC7.png',
 'mask_path': './data/processed/train/masks/obj_01_car/obj_01_car_CC7.png',
 'viewpoint_id': 'CC7',
 'lighting_condition_id': '004',
 'image_id': 'obj_01_car',
 'output_dir': './data/processed/128_v1/train/hints',
 'fov': None,
 'mask_threshold': 0.25,
 'env_map': './data/generate_light_gt_sg/hdrs/004.hdr',
 'pls': [[0, 0, 0]],
 'use_gpu_for_rendering': True,
 'resolution': 128}

In [10]:
radiance_result = utils_gpu.elem_generate_hint(train_split[0])

Rendering radiance hints for ./data/processed/train/images/obj_01_car/obj_01_car_004_CC7.png with viewpoint CC7 and lighting condition 004
Radiance hints already rendered to ./data/processed/128_v1/train/hints/obj_01_car/CC7/004/hint00_diffuse.png
Radiance hints already rendered to ./data/processed/128_v1/train/hints/obj_01_car/CC7/004/hint00_ggx0.05.png
Radiance hints already rendered to ./data/processed/128_v1/train/hints/obj_01_car/CC7/004/hint00_ggx0.13.png
Radiance hints already rendered to ./data/processed/128_v1/train/hints/obj_01_car/CC7/004/hint00_ggx0.34.png
>> Inference with model on 2 image pairs


100%|██████████| 2/2 [00:00<00:00,  2.37it/s]


 init edge (0*,1*) score=19.84580421447754
 init loss = 0.0017114599468186498
Global alignement - optimizing for:
['pw_poses', 'im_depthmaps', 'im_poses', 'im_focals']


100%|██████████| 300/300 [00:04<00:00, 69.31it/s, lr=1.27413e-06 loss=0.00125564]


Mesh reconstructed with fov: 33.81208801269531
Data are loaded, start creating Blender stuff
glTF import finished in 0.25s
> /data2/code/diffusion-project/DiLightNet/demo/render_hints.py(73)configure_blender()
     72             # Enable the GPU device
---> 73             for device in preferences.devices:
     74                 if device.type == 'CUDA':



Rendering Hints: 100%|██████████| 1/1 [00:02<00:00,  2.86s/it]


In [11]:
# directory structure -> list of data

step1_datadict_train_list: List[Dict[str, str]] = []
step1_datadict_eval_list: List[Dict[str, str]] = []

for each_class_name in dataset_def.class_name_list:
    for each_view_point in dataset_def.view_point_list:
        for each_light_condition in dataset_def.light_condition_list:

            image_path = DataSetDefinition.get_image_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                light_condition=each_light_condition,
                view_point=each_view_point,
            )
            mask_path = DataSetDefinition.get_mask_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                view_point=each_view_point,
            )

            parsed_name = DataSetDefinition.get_image_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                light_condition=each_light_condition,
                view_point=each_view_point,
                parsed=True,
            )

            if parsed_name in dataset_def.eval_split:
                step1_datadict_eval_list.append(
                    {
                        "key": f"{each_class_name}_{each_light_condition}_{each_view_point}",
                        "image_path": image_path,
                        "mask_path": mask_path,
                    }
                )
            else:
                step1_datadict_train_list.append(
                    {
                        "key": f"{each_class_name}_{each_light_condition}_          {each_view_point}",
                        "image_path": image_path,
                        "mask_path": mask_path,
                    }
                )